# Preprocessing the Data - The Other Way

In this intriguing assignment, the goal was to exchange our swords - or rather, our feature extraction methods - with a fellow student. This twist meant that I delved into the techniques my peer employed in their dataset, and they ventured into the methods I used in my previous assignment (for reference, see [PreprocessingTheData.ipynb](PreprocessingTheData.ipynb)). It's a fantastic opportunity to step outside our comfort zones and see how our data dances with different partners.

## Feature Extraction Methods

Feature extraction is like finding the magic keys that unlock the most information or the most meaningful signals from our dataset. In this section, I'm going to embark on a journey with the following magical tools:

### 1. Autoencoder

Autoencoders are neural networks designed to learn an efficient representation (encoding) for data, typically for the purpose of dimensionality reduction. They work by compressing the input into a lower-dimensional code and then reconstructing the output from this representation. The beauty of autoencoders lies in their ability to capture the most salient features of the data automatically.

An autoencoder consists of two main parts: the encoder, which compresses the input data, and the decoder, which reconstructs the data from the compressed form. The goal is to minimize the difference between the original input and the reconstructed output, thus forcing the autoencoder to learn a compact representation of the data.

### 2. Mutual Information

Mutual information is a measure from information theory, describing the amount of information obtained about one random variable through observing another random variable. In feature selection, we use it to quantify the "amount of information" a feature gives us about the outcome. It's a powerful way to identify features that contribute most to predicting the target variable, enabling us to focus on relevant data and potentially improve model performance.

### 3. PCA (Principal Component Analysis)

PCA is a statistical technique that converts high-dimensional data into a lower-dimensional form (components) while retaining as much of the variance in the dataset as possible. It's like finding the best angles to view a gemstone, revealing its structure in fewer dimensions. PCA helps in reducing the dimensionality of the data, improving computational efficiency, and sometimes even improving model performance by eliminating noise and redundant features.

### 4. Random Forest

While primarily a machine learning model, Random Forest can also be leveraged for feature extraction. By training a Random Forest model, we can inspect the feature importances attributed by the model, giving us insights into which features it finds most predictive. This method is particularly useful for understanding feature importance in non-linear and complex datasets, helping to highlight which features are most influential in predicting the outcome.

Each of these methods brings a unique perspective on how to unlock the secrets within our data. By experimenting with and understanding these techniques, we gain valuable skills and insights that can be applied across a wide range of data science challenges.

# Initial data preprocessing
Similarly like we had in the previous notebook we would load the EP1.01.txt data as it represents in the best way the rest of the data. Also we would process the "data" colum from string (With "," delimiter) to an array of floats that represent the signals.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
data_path = Path('../data')
file_path = data_path / 'EP1.01.txt'

columns = ['id', 'event_id', 'device', 'channel', 'code', 'size', 'data']
data = pd.read_csv(file_path, delimiter='\t', header=None, names=columns)

In [3]:
data.head()

id  event_id device channel  code  size  \
0  67635     67635     EP     AF3     6   260   
1  67636     67635     EP      F7     6   260   
2  67637     67635     EP      F3     6   260   
3  67638     67635     EP     FC5     6   260   
4  67639     67635     EP      T7     6   260   

                                                data  
0  4395.384615,4382.564102,4377.435897,4387.17948...  
1  4489.230769,4475.384615,4474.358974,4486.66666...  
2  4538.461538,4528.717948,4524.615384,4526.15384...  
3  4207.692307,4205.641025,4200.512820,4194.35897...  
4  4497.948717,4498.461538,4494.871794,4497.94871...

In [4]:
def parse_data(data: str) -> list:
    return list(map(float, data.split(',')))

In [5]:
data.loc[:, 'data'] = data['data'].apply(parse_data)

In [6]:
data.head()

id  event_id device channel  code  size  \
0  67635     67635     EP     AF3     6   260   
1  67636     67635     EP      F7     6   260   
2  67637     67635     EP      F3     6   260   
3  67638     67635     EP     FC5     6   260   
4  67639     67635     EP      T7     6   260   

                                                data  
0  [4395.384615, 4382.564102, 4377.435897, 4387.1...  
1  [4489.230769, 4475.384615, 4474.358974, 4486.6...  
2  [4538.461538, 4528.717948, 4524.615384, 4526.1...  
3  [4207.692307, 4205.641025, 4200.51282, 4194.35...  
4  [4497.948717, 4498.461538, 4494.871794, 4497.9...

# 1. Autoencoder
We'll break down the process into the following steps
1. Preprocess the data - Normalize the EEG signal to have zero mean and unit variance
2. Build the autoencoder - Define Autoencoder architecture
3. Train the Autoencoder
4. Feature extraction

## 1. Preprocessing the Data 

In [7]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()

In [8]:
data.loc[:, 'normalized_data'] = data['data'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)).flatten())

In [9]:
data.head()

id  event_id device channel  code  size  \
0  67635     67635     EP     AF3     6   260   
1  67636     67635     EP      F7     6   260   
2  67637     67635     EP      F3     6   260   
3  67638     67635     EP     FC5     6   260   
4  67639     67635     EP      T7     6   260   

                                                data  \
0  [4395.384615, 4382.564102, 4377.435897, 4387.1...   
1  [4489.230769, 4475.384615, 4474.358974, 4486.6...   
2  [4538.461538, 4528.717948, 4524.615384, 4526.1...   
3  [4207.692307, 4205.641025, 4200.51282, 4194.35...   
4  [4497.948717, 4498.461538, 4494.871794, 4497.9...   

                                     normalized_data  
0  [0.8441558461291976, 0.5194805129026889, 0.389...  
1  [0.4296875010711574, 0.2187499966674693, 0.203...  
2  [0.945205480183887, 0.684931497701271, 0.57534...  
3  [0.3440860178980216, 0.3010752656607565, 0.193...  
4  [0.43749997714844824, 0.44999998903124094, 0.3...

What we essentially did here was to transform the data such that its range would be between 0 and 1. This is a common preprocessing step in machine learning, as it helps to ensure that all features contribute equally to the model and are on a similar scale. Normalizing the data can also help to improve the convergence of optimization algorithms and the overall performance of the model.

## 2. Building the autoencoder

Building an autoencoder involves defining both an encoder and a decoder. The encoder compresses the input data into a lower-dimensional representation, and the decoder attempts to reconstruct the original input data from this compressed representation. Autoencoders are trained to minimize the reconstruction error, often using mean squared error (MSE) or a similar loss function.

In [10]:
import torch
from torch import nn

In [11]:
class SimpleAutoencoder(nn.Module):
    def __init__(self, input_size):
        super(SimpleAutoencoder, self).__init__()
        # Encoder: Linear layer reducing dimensionality from input_size to a compressed representation
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),  # Compressed representation
            nn.ReLU()
        )
        # Decoder: Linear layer reconstructing the original input from the compressed representation
        self.decoder = nn.Sequential(
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

Check the length of all the inputs we have in column "size"

In [12]:
data['size'].unique()

array([260, 256, 264, 261, 252, 257, 248, 272, 268, 265, 249, 253, 204,
       320, 269, 276, 244, 273, 196, 228, 292, 224, 280, 212, 232, 240,
       284, 136])

Looks like we have lots of different sizes, lets find the mean size and cut everything to it.

In [13]:
data['size'].describe()

count    910476.000000
mean        259.511901
std           2.319790
min         136.000000
25%         260.000000
50%         260.000000
75%         260.000000
max         320.000000
Name: size, dtype: float64

In [14]:
filtered_data = data[data['size'] == 260]
filtered_data['size'].describe()

count    606214.0
mean        260.0
std           0.0
min         260.0
25%         260.0
50%         260.0
75%         260.0
max         260.0
Name: size, dtype: float64

In [15]:
input_size = filtered_data['size'].unique()[0]
autoencoder = SimpleAutoencoder(input_size)

## 3. Training the Autoencoder
We'll start by creating a dataloader from the dataset we have

In [16]:
filtered_data['normalized_data']

0         [0.8441558461291976, 0.5194805129026889, 0.389...
1         [0.4296875010711574, 0.2187499966674693, 0.203...
2         [0.945205480183887, 0.684931497701271, 0.57534...
3         [0.3440860178980216, 0.3010752656607565, 0.193...
4         [0.43749997714844824, 0.44999998903124094, 0.3...
                                ...                        
908217    [0.8699187022803727, 0.8943089535296451, 0.666...
908218    [0.7868852493079856, 0.7540983645861274, 0.344...
908219    [0.6814159374500832, 0.707964600700933, 0.5044...
908220    [0.846715338856626, 0.9270073008284925, 0.7372...
908221    [0.9819819824567873, 0.9909909824446146, 0.756...
Name: normalized_data, Length: 606214, dtype: object

In [17]:
from torch.utils.data import Dataset, DataLoader
import torch

In [18]:
class EEGDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.data = list(dataframe['normalized_data'])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = torch.tensor(sample, dtype=torch.float)
        return sample
    
dataset = EEGDataset(filtered_data)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

Now we will set our device agnostic and run some epochs on the data and train the model

In [19]:
device = 'cpu'
# Check if device is mps or Cuda
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'

autoencoder.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

num_epochs = 20
loss = None
for epoch in range(num_epochs):
    for data in dataloader:
        data = data.to(device)
        reconstructed = autoencoder(data)
        loss = criterion(reconstructed, data)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.0155
Epoch [2/20], Loss: 0.0203
Epoch [3/20], Loss: 0.0138
Epoch [4/20], Loss: 0.0226
Epoch [5/20], Loss: 0.0192
Epoch [6/20], Loss: 0.0130
Epoch [7/20], Loss: 0.0195
Epoch [8/20], Loss: 0.0170
Epoch [9/20], Loss: 0.0107
Epoch [10/20], Loss: 0.0133
Epoch [11/20], Loss: 0.0188
Epoch [12/20], Loss: 0.0139
Epoch [13/20], Loss: 0.0135
Epoch [14/20], Loss: 0.0210
Epoch [15/20], Loss: 0.0114
Epoch [16/20], Loss: 0.0132
Epoch [17/20], Loss: 0.0133
Epoch [18/20], Loss: 0.0155
Epoch [19/20], Loss: 0.0199
Epoch [20/20], Loss: 0.0163


## 4. Extracting Features

In [20]:
features = []
for data in DataLoader(dataset, batch_size=64, shuffle=False):
    data = data.to(device)
    feature = autoencoder.encoder(data)
    features.append(feature.cpu().detach().numpy())

# Concatenate all features to get the full feature matrix
features = np.concatenate(features, axis=0)

In [21]:
features

array([[0.7446833 , 1.0197172 , 0.        , ..., 0.6929073 , 0.8318233 ,
        1.1829625 ],
       [1.328233  , 0.5220584 , 0.        , ..., 0.622978  , 0.703767  ,
        1.3462029 ],
       [0.83387864, 1.1196021 , 0.        , ..., 0.9277895 , 0.75806546,
        1.1536936 ],
       ...,
       [1.1984265 , 0.6803893 , 0.        , ..., 0.72096837, 0.5486329 ,
        1.2915742 ],
       [0.982052  , 0.91261685, 0.        , ..., 0.6064304 , 0.6889183 ,
        1.4587922 ],
       [1.0606124 , 0.87201345, 0.        , ..., 0.5124967 , 0.61672145,
        1.3320158 ]], dtype=float32)

In [22]:
import pandas as pd
features_df = pd.DataFrame(features)

features_df.index = filtered_data.index

features_df = features_df.add_prefix('feature_')

enhanced_data = pd.concat([filtered_data, features_df], axis=1)

In [23]:
enhanced_data.head()

id  event_id device channel  code  size  \
0  67635     67635     EP     AF3     6   260   
1  67636     67635     EP      F7     6   260   
2  67637     67635     EP      F3     6   260   
3  67638     67635     EP     FC5     6   260   
4  67639     67635     EP      T7     6   260   

                                                data  \
0  [4395.384615, 4382.564102, 4377.435897, 4387.1...   
1  [4489.230769, 4475.384615, 4474.358974, 4486.6...   
2  [4538.461538, 4528.717948, 4524.615384, 4526.1...   
3  [4207.692307, 4205.641025, 4200.51282, 4194.35...   
4  [4497.948717, 4498.461538, 4494.871794, 4497.9...   

                                     normalized_data  feature_0  feature_1  \
0  [0.8441558461291976, 0.5194805129026889, 0.389...   0.744683   1.019717   
1  [0.4296875010711574, 0.2187499966674693, 0.203...   1.328233   0.522058   
2  [0.945205480183887, 0.684931497701271, 0.57534...   0.833879   1.119602   
3  [0.3440860178980216, 0.3010752656607565, 0.193...   0.579244   0.710636   
4  [0.43749997714844824, 0.44999998903124094, 0.3...   0.755086   0.806043   

   feature_2  feature_3  feature_4  feature_5  feature_6  feature_7  \
0        0.0        0.0   0.759063   0.941369   1.030296   1.128954   
1        0.0        0.0   0.677633   0.853899   0.722285   0.706722   
2        0.0        0.0   0.935851   0.815357   1.122367   1.101907   
3        0.0        0.0   0.893642   0.679314   0.980356   0.703967   
4        0.0        0.0   0.689965   0.589381   1.001275   0.734863   

   feature_8  feature_9  feature_10  feature_11  
0        0.0   0.692907    0.831823    1.182963  
1        0.0   0.622978    0.703767    1.346203  
2        0.0   0.927790    0.758065    1.153694  
3        0.0   0.574136    0.882578    1.157292  
4        0.0   0.867773    0.835718    0.840352

# 2. Mutual Information
In this part we will check which of the features (That we selected from the autoencoder) are the most important for the target variable.

In [24]:
from sklearn.feature_selection import mutual_info_regression

# Selecting the features and target variable
X = enhanced_data.drop(columns=['id', 'event_id', 'device', 'channel', 'code', 'size', 'data', 'normalized_data'])
y = enhanced_data['code']

# Calculating the mutual information
mi = mutual_info_regression(X, y)
mi = pd.Series(mi, index=X.columns)
mi = mi.sort_values(ascending=False)
mi

feature_5     0.002282
feature_11    0.001673
feature_6     0.001060
feature_0     0.000915
feature_1     0.000806
feature_10    0.000028
feature_2     0.000000
feature_3     0.000000
feature_4     0.000000
feature_7     0.000000
feature_8     0.000000
feature_9     0.000000
dtype: float64

From these results we can clearly see that the feature_0 is the most important feature for the target variable. This is a powerful insight, as it allows us to focus on the most relevant features when building our predictive model. By understanding which features are most informative, we can potentially improve the performance of our model and gain deeper insights into the underlying patterns in the data.

# 3. PCA (Principal Component Analysis)
In this part we will use PCA to reduce the dimensionality of the data and extract the most important components. This will help us to visualize the data in a lower-dimensional space and identify the key patterns and relationships within the data.

In [28]:
from sklearn.decomposition import PCA

# Fit PCA to the features
pca = PCA(n_components=11) # We have 0-9 numbers shown to subject and (-1) for random signal (Total 11)
pca_features = pca.fit_transform(X)

# Create a DataFrame with the PCA features
pca_df = pd.DataFrame(data=pca_features, columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11'])
pca_df.index = X.index

# Concatenate the PCA features with the original data
pca_data = pd.concat([enhanced_data, pca_df], axis=1)
pca_data.head()

id  event_id device channel  code  size  \
0  67635     67635     EP     AF3     6   260   
1  67636     67635     EP      F7     6   260   
2  67637     67635     EP      F3     6   260   
3  67638     67635     EP     FC5     6   260   
4  67639     67635     EP      T7     6   260   

                                                data  \
0  [4395.384615, 4382.564102, 4377.435897, 4387.1...   
1  [4489.230769, 4475.384615, 4474.358974, 4486.6...   
2  [4538.461538, 4528.717948, 4524.615384, 4526.1...   
3  [4207.692307, 4205.641025, 4200.51282, 4194.35...   
4  [4497.948717, 4498.461538, 4494.871794, 4497.9...   

                                     normalized_data  feature_0  feature_1  \
0  [0.8441558461291976, 0.5194805129026889, 0.389...   0.744683   1.019717   
1  [0.4296875010711574, 0.2187499966674693, 0.203...   1.328233   0.522058   
2  [0.945205480183887, 0.684931497701271, 0.57534...   0.833879   1.119602   
3  [0.3440860178980216, 0.3010752656607565, 0.193...   0.579244   0.710636   
4  [0.43749997714844824, 0.44999998903124094, 0.3...   0.755086   0.806043   

   ...      PCA2      PCA3      PCA4      PCA5      PCA6      PCA7      PCA8  \
0  ... -0.206331 -0.015859 -0.280363  0.249040  0.309885 -0.074663  0.054720   
1  ...  0.456196  0.111508  0.053692 -0.277887  0.332281  0.033797 -0.211985   
2  ... -0.296894 -0.195174 -0.394672  0.017065  0.273719 -0.183157  0.147437   
3  ...  0.030089 -0.139138  0.102751  0.197387 -0.103275  0.012725  0.171528   
4  ... -0.288078 -0.172381  0.239058 -0.086440  0.170820  0.035641  0.198398   

       PCA9         PCA10         PCA11  
0  0.067630 -4.706660e-09  2.172482e-09  
1  0.055549  3.166768e-10  3.604087e-09  
2  0.116440  3.210145e-06  1.558662e-06  
3  0.097001  3.798111e-06 -1.317410e-06  
4  0.106037 -2.061212e-11  2.440442e-12  

[5 rows x 31 columns]

In [32]:
# Plot the explained variance ratio
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance Ratio by Number of Components')
plt.show()

We see that the first 11 components explain almost 100% of the variance in the data. This means that we can effectively represent the data in a lower-dimensional space while retaining most of the information. By using PCA, we can reduce the dimensionality of the data, improve computational efficiency, and potentially enhance the performance of our predictive model.

# 4. Random Forest
The final feature extraction method we'll explore is Random Forest. We'll train a Random Forest model on the data and examine the feature importances attributed by the model. This will help us to identify which features are most influential in predicting the target variable and gain insights into the underlying patterns in the data.

In [33]:
from sklearn.ensemble import RandomForestClassifier

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances
feature_importances = rf.feature_importances_
feature_importances = pd.Series(feature_importances, index=X.columns)
feature_importances = feature_importances.sort_values(ascending=False)
feature_importances

feature_9     0.111331
feature_1     0.111257
feature_4     0.111220
feature_5     0.111200
feature_0     0.111180
feature_6     0.111121
feature_11    0.111064
feature_7     0.111047
feature_10    0.110581
feature_2     0.000000
feature_3     0.000000
feature_8     0.000000
dtype: float64

In [34]:
# Plot the feature importances
plt.figure(figsize=(10, 6))
feature_importances.plot(kind='bar')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importances from Random Forest')
plt.show()

The feature importances from the Random Forest model provide valuable insights into which features are most predictive of the target variable. By understanding the importance of each feature, we can focus on the most relevant data and potentially improve the performance of our predictive model. This method is particularly useful for identifying key patterns and relationships in complex datasets, helping us to extract meaningful information and make informed decisions.

We can see in our case that the features 2, 3 and 8 have 0 value for the importance, this means that they are not important for the target variable and we can remove them from the dataset.

So if we would take the information from the random forest and the mutual information we can clearly see that the features 2, 3 and 8 are not important for the target variable and we can remove them from the dataset without any loss of information. 